In [1]:
# Must be included at the beginning of each new notebook.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Data_Mining_GlobalTerrorism').getOrCreate()

# Importing complete dataset

In [2]:
df = spark.read.csv('Complete_dataset.csv', header=True, inferSchema=True)

In [3]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Date: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Attack: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Individual: integer (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Ishostkid: double (nullable = true)



In [4]:
from pyspark.sql import *
spark = SparkSession.builder.appName('Final1').getOrCreate()
# Print data columns.
df.columns

['Year',
 'Month',
 'Date',
 'Country',
 'Region',
 'Province',
 'City',
 'Attack',
 'Target',
 'Nationality',
 'Group',
 'Individual',
 'Weapon',
 'Ishostkid']

# Data Transformation - encoding

In [5]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

# Goal 2

In [10]:
#indexing

Country_indexer = StringIndexer(inputCol='Country',outputCol='CountryIndex')
Region_indexer = StringIndexer(inputCol='Region',outputCol='RegionIndex')
Province_indexer = StringIndexer(inputCol='Province',outputCol='ProvinceIndex')
City_indexer = StringIndexer(inputCol='City',outputCol='CityIndex')
Target_indexer = StringIndexer(inputCol='Target',outputCol='TargetIndex')
Nationality_indexer = StringIndexer(inputCol='Nationality',outputCol='NationalityIndex')
Group_indexer = StringIndexer(inputCol='Group',outputCol='GroupIndex')
Weapon_indexer = StringIndexer(inputCol='Weapon',outputCol='WeaponIndex')
Attack_indexer = StringIndexer(inputCol='Attack',outputCol='label')

#encoding

Country_encoder = OneHotEncoder(inputCol='CountryIndex',outputCol='CountryVec')
Region_encoder = OneHotEncoder(inputCol='RegionIndex',outputCol='RegionVec')
Province_encoder = OneHotEncoder(inputCol='ProvinceIndex',outputCol='ProvinceVec')
City_encoder = OneHotEncoder(inputCol='CityIndex',outputCol='CityVec')
Target_encoder = OneHotEncoder(inputCol='TargetIndex',outputCol='TargetVec')
Nationality_encoder = OneHotEncoder(inputCol='NationalityIndex',outputCol='NationalityVec')
Group_encoder = OneHotEncoder(inputCol='GroupIndex',outputCol='GroupVec')
Weapon_encoder = OneHotEncoder(inputCol='WeaponIndex',outputCol='WeaponVec')

label_encoder = OneHotEncoder(inputCol='label',outputCol='label')



In [11]:
# Now we can assemble all of this as one vector in the features column. 
assembler = VectorAssembler(inputCols=['Year','CountryVec',
 'RegionVec',
 'ProvinceVec',
 'CityVec',
 'TargetVec',
 'NationalityVec',
 'GroupVec',
 'WeaponVec'],outputCol='features')

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
from pyspark.ml import Pipeline

In [14]:
log_reg_attack = LogisticRegression(featuresCol='features',labelCol='label')

In [19]:
pipeline = Pipeline(stages=[Country_indexer,Region_indexer,Province_indexer,City_indexer,Target_indexer,Nationality_indexer,Group_indexer,
                           Weapon_indexer,Attack_indexer,Country_encoder,Region_encoder,Province_encoder,City_encoder,Target_encoder,Nationality_encoder,Group_encoder,Weapon_encoder,
                           assembler,log_reg_attack])

In [20]:
# Train/test split. 
train_attack_data, test_attack_data = df.randomSplit([0.7,.3])

In [ ]:
# Note pipeline. Call it as you would call a machine learning object.
fit_model = pipeline.fit(train_titanic_data)

In [ ]:
# Transform test data. 
results = fit_model.transform(test_titanic_data)

In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[Year_indexer,Country_indexer,Region_indexer,Province_indexer,City_indexer,Target_indexer,Nationality_indexer,Group_indexer,Weapon_indexer,Attack_indexer,Year_encoder,Country_encoder,Region_encoder,Province_encoder,City_encoder,Target_encoder,Nationality_encoder,Group_encoder,Weapon_encoder,assembler])

# Pipeline

In [8]:
pipeline_model = pipeline.fit(df)
pipe_df = pipeline_model.transform(df)
pipe_df = pipe_df.select('label','features')
pipe_df.describe()

DataFrame[summary: string, label: string]

# Splitting of data

In [9]:
train_data, test_data = pipe_df.randomSplit([0.7,0.3])
print("Training Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 78215
Test Dataset Count: 33641


# ML Algorithms

In [10]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [11]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# generate the train/test split.
(train_data, test_data) = pipe_df.randomSplit([0.7, 0.3])

# instantiate the base classifier.
lr = LogisticRegression(maxIter=5, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(train_data)

# score the model on test data.
predictions = ovrModel.transform(test_data)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test accuracy = %g" % (1.0 - accuracy))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/int

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44513)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-e323a51f41d2>", line 24, in <module>
    accuracy = evaluator.evaluate(predictions)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 69, in evaluate
    return self._evaluate(dataset)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/ml/evaluation.py", line 99, in _evaluate
    return self._java_obj.evaluate(dataset._jdf)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, 

Py4JError: An error occurred while calling o1436.evaluate